In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Sashank Ram\\Documents\\End-to-end-ML-Final-with-MLFlo\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Sashank Ram\\Documents\\End-to-end-ML-Final-with-MLFlo'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [24]:

from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [25]:
from src.mlProject.constants import *

In [30]:
from src.mlProject.constants._init_ import CONFIG_FILE_PATH
from src.mlProject.constants._init_ import PARAMS_FILE_PATH
from src.mlProject.constants._init_ import SCHEMA_FILE_PATH

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params =read_yaml(params_filepath)
            self.schema =read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [31]:
import os
import urllib.request as request
import zipfile
from src.mlProject.utils._init_ import logger
from src.mlProject.utils.common import get_size

In [40]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        os.makedirs(Path(self.config.local_data_file).parent, exist_ok=True)


        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,             # <-- use source_URL
                filename=str(self.config.local_data_file)    # <-- save to local path
        )
            logger.info(f"{filename} downloaded with headers:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [41]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-31 22:49:51,920: INFO: common: yaml file:config\config.yaml loaded successfully]
[2025-08-31 22:49:51,926: INFO: common: yaml file:params.yaml loaded successfully]
[2025-08-31 22:49:51,927: INFO: common: yaml file:schema.yaml loaded successfully]
[2025-08-31 22:49:51,933: INFO: common: created directory at:artifacts]
[2025-08-31 22:49:51,933: INFO: common: created directory at:artifacts/data_ingestion]
[2025-08-31 22:49:53,849: INFO: 1371385911: artifacts\data_ingestion\data.zip downloaded with headers:
Connection: close
Content-Length: 376518
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e1ca5ca99103771599be6d058029827254429c0cfc1672650fe01a3963fe2ce1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EF33:B93BA:61504:D3CB8:68B48438
Accept-Ranges: bytes
Date: Sun, 31 Aug 2025 17:1